In [ ]:
import pandas as pd
import numpy as np
import math
from datetime import datetime,timezone
import glob

import seaborn as sns
import matplotlib.pyplot  as plt

# Init Variable

In [ ]:
dateCols=['open_datetime','close_datetime','imported_at']

# Load data from CSV

In [ ]:
print("Load data from CSV")
df_all=None

listCSVFile = glob.glob("data/incident_*.csv")
for file in listCSVFile:
  print(file)  
  df=pd.read_csv(file,parse_dates=dateCols)
  print(df.shape)
  df_all=pd.concat([df_all,df]) 
df_all.info()

# Clean Data & Transform Data

In [ ]:
df_all=df_all.sort_values(by='id')
df_all=df_all.drop_duplicates(subset=['id'],keep='last')


In [ ]:
print("Create Time to Service Cols")

df_all['open_diff_close']=df_all['close_datetime']-df_all['open_datetime']
df_all['open_to_close_hour'] = df_all['open_diff_close'].apply(lambda x:  x.total_seconds() / (60*60) if x is not np.nan else np.nan  )

# remove  hour > 8760 hours (1 year) , it very odd.
# remove  hour <=0  it is nonesense.
df_all=df_all.query('open_to_close_hour>0 and open_to_close_hour<=8760')

In [ ]:
print("Create Period Level to Service")
def service_hour_range(x_hour):
    if x_hour>0 and x_hour<=24:  
      return "day"
    elif x_hour>25 and x_hour<=168:   
      return "week"
    elif x_hour>168 and x_hour<=360:
      return "half-month"
    elif x_hour>360 and x_hour<=720: 
      return "month"
    elif x_hour>720 and x_hour<=2160: 
      return "quarter"
    else: 
      return "year"

df_all['open_to_close_hour_range']=df_all['open_to_close_hour'].apply(service_hour_range)

# Create Label

In [ ]:
#4-Cosmetic=0  3-Minor=1   2-Major=2  1-Critical=3
# map_severity_to_class={4:0,3: 1, 2: 2, 1: 3}
# print(f"Map severity id to LabelCode: {str(map_severity_to_class)}")
# df_all['label_multi_severity'] =df_all['severity_id'].map(map_severity_to_class) 

def map_4to2_serverity(severity_id):
    if severity_id==1 or severity_id==2:
       return 1
    else:
       return 0 
df_all['label_binary_severity'] =df_all['severity_id'].apply(map_4to2_serverity)



In [ ]:
df_all=df_all.drop(columns=['open_datetime','close_datetime','imported_at','open_diff_close'])
df_all[['id','severity_id']] =df_all[['id','severity_id']].astype('int64')


In [ ]:
df_all.info()

# Explore Label

In [ ]:
def plot_label(df_,label_):

    fig , ax = plt.subplots(figsize=(15,5))
    ax =sns.countplot(x=label_, data=df_,)
    for p in ax.patches:
       ax.annotate('{:.1f}'.format(p.get_height()), (p.get_x()+0.25, p.get_height()+0.01))
    plt.title(label_)
    plt.show()
        
# plot_label(df_all,'label_multi_severity')
plot_label(df_all,'label_binary_severity')

In [ ]:
# Explore Feature
##  product_type  as  vertical chart bar ,sort dese
##  incident_type  as horizontal chart bar, sort desc
##  service_type, sla as pie chart 

# Save DataSet

In [ ]:
df_all.to_csv('dataset.csv',index=False)